In [26]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import os
from konlpy.tag import Hannanum, Mecab, Kkma, Komoran, Okt
import pandas as pd
import time

In [27]:
import os
csv_path = os.getenv("HOME") + "/news_crawler/news_data.csv"
df = pd.read_table(csv_path, sep=',')
df.head()

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
4,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,사회


In [28]:
# 정규표현식을 사용하여 한글만 사용
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

0       파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기자 파주시는 ...
1       동영상 뉴스이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소 공...
2       황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...
3       귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...
4       서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 사회적 거리두기에...
                              ...                        
5119    신종 코로나바이러스 감염증코로나 사태 이후 가정의 달 월에도 언택트비대면 신풍속도가...
5120    는 소비자로부터 월 이용료 만만원을 받고 초고속 인터넷을 제공한다 그런 브로드밴드가...
5121    머리를 긁고 있는 오랑우탄 몸을 긁는 행동을 따라 하는 것은 부정적 감정과 관련이 ...
5122    가 오는 일 정식 출시하는 스마트폰 벨벳이 사실상 공짜폰이 될 전망이다 단말기 가격...
5123    이미지제공게티이미지뱅크 이미지제공게티이미지뱅크  전자신문  전자신문인터넷 무단전재 ...
Name: news, Length: 5124, dtype: object

In [29]:
# 중복 샘플 제거
df.drop_duplicates(subset=['news'], inplace=True)

In [30]:
# 불용어 처리
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

In [31]:
# 토크나이즈 성능 비교를 위한 토크나이즈 설정
Mecab_t = Mecab()
Hannanum_t = Hannanum()
Kkma_t = Kkma()
Komoran_t = Komoran(userdic='/tmp/dic.txt')
Okt_t = Okt()

In [32]:
def preprocessing(tokenizer, data):
  text_data = []
  for sentence in data:
    temp_data = tokenizer.morphs(sentence) 
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [33]:
# # 토크나이즈 성능 비교를 위한 토크나이즈 설정
# Mecab_t = Mecab()
# hannanum_t = Hannanum()
# kkma_t = Kkma()
# komoran_t = Komoran(userdic='/tmp/dic.txt')
# okt_t = Okt()
result = pd.DataFrame( index = ['Mecab', 'Hannanum', 'Kkma', 'Komoran', 'Okt'], columns = ['time', 'precision', 'recall', 'f1'])
result

,time,precision,recall,f1
Mecab,NaN,NaN,NaN,NaN
Hannanum,NaN,NaN,NaN,NaN
Kkma,NaN,NaN,NaN,NaN
Komoran,NaN,NaN,NaN,NaN
Okt,NaN,NaN,NaN,NaN


In [34]:
prev = time.time()
text_data = preprocessing(Mecab_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

result.loc['Mecab','time'] = time.time() - prev
result.loc['Mecab','precision'] = metrics.precision_score(y_test, y_pred, average='weighted')
result.loc['Mecab','recall'] = metrics.recall_score(y_test, y_pred, average='weighted')
result.loc['Mecab','f1'] = metrics.f1_score(y_test, y_pred, average='weighted')


              precision    recall  f1-score   support

       IT/과학       0.90      0.74      0.81       236
          사회       0.79      0.91      0.85       422
       생활/문화       0.81      0.76      0.79       341

    accuracy                           0.82       999
   macro avg       0.83      0.80      0.81       999
weighted avg       0.82      0.82      0.82       999



In [35]:
prev = time.time()
text_data = preprocessing(Hannanum_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

result['Hannanum','time'] = time.time() - prev
result['Hannanum','precision'] = metrics.precision_score(y_test, y_pred, average='weighted')
result['Hannanum','recall'] = metrics.recall_score(y_test, y_pred, average='weighted')
result['Hannanum','f1'] = metrics.f1_score(y_test, y_pred, average='weighted')


              precision    recall  f1-score   support

       IT/과학       0.90      0.64      0.75       236
          사회       0.79      0.91      0.85       422
       생활/문화       0.79      0.79      0.79       341

    accuracy                           0.81       999
   macro avg       0.83      0.78      0.80       999
weighted avg       0.82      0.81      0.81       999



In [36]:
prev = time.time()
text_data = preprocessing(Kkma_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

result.loc['Kkma','time'] = time.time() - prev
result.loc['Kkma','precision'] = metrics.precision_score(y_test, y_pred, average='weighted')
result.loc['Kkma','recall'] = metrics.recall_score(y_test, y_pred, average='weighted')
result.loc['Kkma','f1'] = metrics.f1_score(y_test, y_pred, average='weighted')


              precision    recall  f1-score   support

       IT/과학       0.90      0.75      0.82       236
          사회       0.80      0.91      0.85       422
       생활/문화       0.81      0.77      0.79       341

    accuracy                           0.82       999
   macro avg       0.84      0.81      0.82       999
weighted avg       0.83      0.82      0.82       999



In [37]:
prev = time.time()
text_data = preprocessing(Komoran_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

result.loc['Komoran','time'] = time.time() - prev
result.loc['Komoran','precision'] = metrics.precision_score(y_test, y_pred, average='weighted')
result.loc['Komoran','recall'] = metrics.recall_score(y_test, y_pred, average='weighted')
result.loc['Komoran','f1'] = metrics.f1_score(y_test, y_pred, average='weighted')


              precision    recall  f1-score   support

       IT/과학       0.90      0.74      0.81       236
          사회       0.80      0.91      0.85       422
       생활/문화       0.81      0.77      0.79       341

    accuracy                           0.82       999
   macro avg       0.83      0.81      0.82       999
weighted avg       0.83      0.82      0.82       999



In [38]:
prev = time.time()
text_data = preprocessing(Okt_t, df['news'])
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

result.loc['Okt','time'] = time.time() - prev
result.loc['Okt','precision'] = metrics.precision_score(y_test, y_pred, average='weighted')
result.loc['Okt','recall'] = metrics.recall_score(y_test, y_pred, average='weighted')
result.loc['Okt','f1'] = metrics.f1_score(y_test, y_pred, average='weighted')


              precision    recall  f1-score   support

       IT/과학       0.91      0.70      0.79       236
          사회       0.78      0.93      0.85       422
       생활/문화       0.83      0.76      0.79       341

    accuracy                           0.82       999
   macro avg       0.84      0.80      0.81       999
weighted avg       0.83      0.82      0.82       999



In [58]:
result

,time,precision,recall,f1
Mecab,6.05368,0.823619,0.818819,0.817515
Hannanum,163.272,0.816591,0.808809,0.805673
Kkma,1220.94,0.828254,0.823824,0.822678
Komoran,78.5188,0.825295,0.820821,0.81943
Okt,180.024,0.826798,0.817818,0.815507


형태소 분석기에 따른 성능 변화
======================
- 형태소분석기의 종류에 따른 accuracy는 오차범위 내에서 비슷한 수준임
- 다만, 시간 효율적인 측면에서 Mecab의 효율성이 압도적이며, Komoran 역시 나머지 분석기에 비해 빠른 속도를 보임
- Kkma 분석기가 오차범위 내에서 가장 높은 정확도를 보여주나, 처리 속도가 크게 늦기 때문에 효율적이지 못함

- 데이터 관점
    - 분석기 종류에 상관없이 IT분야의 데이터는 precision에 비해 recall값이 많이 낮으며, 사회분야의 데이터는 반대의 경향성을 보임
    - 이는 TF-IDF 분류의 특성상 특정 분야에서만 자주 쓰이는 token이 많을수록 정확도가 높아지는 현상 때문이다.
    - 반대로 사회 분야는 많은 분야를 포괄적으로 다루기 때문에 사회 뉴스에서만 빈출되는 단어가 다른 분야에 비해 적어 정확도가 낮아지는 것임